In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

In [3]:

df = pd.read_excel('all-countries-2017-2023.xlsx', skiprows=11, usecols='A:W')

In [4]:
df.columns

Index(['Country', 'Level 1 Name', 'Area', 'Area ID', 'Analysis Name',
       'Date of Analysis', 'Country Population', '#', '% of total county Pop',
       'Area Phase', 'Analysis Period', '#.1', '%', '#.2', '%.1', '#.3', '%.2',
       '#.4', '%.3', '#.5', '%.4', '#.6', '%.5'],
      dtype='object')

In [5]:
#rename column '#.1', '%', '#.2', '%.1',
df.rename(columns={ 'Country Population': 'population',
                   '#': 'population_analyzed', '% of total county Pop': 'pop_per',
                   '#.1':'phase1_abs', '%':'phase1_per', 
                   '#.2':'#phase2_abs', '%.1':'phase2_abs',
                    '#.3':'phase3_abs', '%.2':'phase3_per',
                    '#.4':'phase4_abs', '%.3':'phase4_per',
                    '#.5':'phase5_abs', '%.4':'phase5_per',
                    '#.6':'phase3plus_abs', '%.5':'phase3plus_per',
                    'Date of Analysis':'date'}, inplace=True)


In [6]:
df.shape

(11100, 23)

In [15]:
#remove rows with NaN values in the 'Country' column
df = df.dropna(subset=['Country'])

#make a seperate df full country data
df_countries = df[df['Area'].isna()]

#convert date to datetime (mm-yyyy)
df_countries['date'] = df_countries['date'].str.replace(' ', '')

df_countries['date'] = pd.to_datetime(df_countries['date'])

/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_18056/2683068199.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_countries['date'] = pd.to_datetime(df_countries['date'])
/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_18056/2683068199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['date'] = pd.to_datetime(df_countries['date'])


In [16]:
#keep only rows where country contains the term "acute food insecurity"
df_countries = df_countries[df_countries['Country'].str.contains('Acute Food Insecurity')]
df_countries['Country'] = df_countries['Country'].str.split(':').str[0]


In [17]:
#keep only Country, population, phase3plus_abs, phase3plus_per, date
df_countries = df_countries[['Country', 'population', 'phase3plus_abs', 'phase3plus_per', 'date']]
df_countries

,Country,population,phase3plus_abs,phase3plus_per,date
0,Afghanistan,43100596.0,16973664.0,0.41,2022-09-01
47,Afghanistan,41727303.0,19650163.0,0.47,2022-03-01
94,Afghanistan,41727304.0,18844453.0,0.47,2021-10-01
141,Afghanistan,32890171.0,10938928.0,0.35,2021-03-01
188,Afghanistan,32890171.0,11149264.0,0.36,2020-09-01
...,...,...,...,...,...
10748,Zambia,17609178.0,913166.0,0.13,2018-06-01
10815,Zimbabwe,15557469.0,2605477.0,0.27,2020-10-01
10885,Zimbabwe,14645000.0,4341420.0,0.45,2020-02-01
10955,Zimbabwe,14645000.0,2294605.0,0.25,2019-06-01


In [18]:
# load https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv
# to get the country codes
url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv'
df_country_codes = pd.read_csv(url)
df_country_codes = df_country_codes[['name', 'alpha-3']]
df_country_codes

,name,alpha-3
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
...,...,...
244,Wallis and Futuna,WLF
245,Western Sahara,ESH
246,Yemen,YEM
247,Zambia,ZMB


In [19]:
#merge df_countries with df_country_codes
df_countries = df_countries.merge(df_country_codes, left_on='Country', right_on='name')
df_countries = df_countries.drop(columns=['name'])

#rename alpha-3 to country_code
df_countries.rename(columns={'alpha-3':'country_code'}, inplace=True)



Country            0
population         0
phase3plus_abs    18
phase3plus_per    18
date               0
country_code       0
dtype: int64

In [20]:
#save as csv
df_countries.to_csv('acutefoodinsecurity.csv', index=False)

In [21]:
# Read csv
#fs='https://hapi.humdata.org/api/v1/food/food-security?ipc_phase=3%2B&ipc_type=current&location_code=YEM&output_format=json&app_identifier=Y2dlbGlsX2xzZTpjLmFiZGVsZ2VsaWxAbHNlLmFjLnVr&limit=10000&offset=0'
#fs_df = pd.json_normalize(requests.get(fs).json()['data'])